<a href="https://colab.research.google.com/github/hlapin/DHTeaching/blob/master/kraken_gpu_experiment_text_recognition_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

This notebook walks through the process of training text-recognition and segmentaiton models using kraken OCR software. (https://github.com/mittagessen/kraken)    


## GPU
The notebook assumes that your Colab notebook is linked to Google Drive (instructions below) and that your google drive has the following structure:   

Use the `runtime` menu to select GPU rather than standard runtime

## Files

````
+-- Google Drive / My Drive
|   +-- kraken_project_folder
    |    +-- <your_project_name>
         |   +-- recognition_training_xml
         |   +-- seg_training_xml
         |   +-- trained_recognition_models
         |   +-- trained_seg_models                      
    |    +-- kraken
         |   +-- seg_models
         |   +-- recognition_models 
````

# Get kraken using pip
Conda did not work -- problem with virtual environments in Colab


### Skip this step  
## Required libraries
   
Older versions of kraken required libraries external libraries        
    as of 7/6/21  unnecessary
  

In [ ]:
!apt install libpangocairo-1.0 libxml2 libblas3 liblapack3 python3-dev python3-pip

### Skip this step  
## Required versions of Python libraries


Get specific versions of libraries required by Kraken
   as of 7/6/21 this was not required
````
WARNING:root:scikit-learn version 0.22.2.post1 is not supported. Minimum required version: 0.17. Maximum required version: 0.19.2. Disabling scikit-learn conversion API.
WARNING:root:TensorFlow version 2.4.1 detected. Last version known to be fully compatible is 2.3.1 .
````


In [ ]:
!pip3 install scikit-learn==0.19.2
!pip3 install TensorFlow===2.3.1

## Install kraken from github using PIP


In [ ]:
!pip install git+https://github.com/mittagessen/kraken.git

Reassure myself that it is installed

In [ ]:
# /check version 
!kraken --version

In [ ]:
!which kraken #directory of main installation
!~/.kraken # any other kraken-specific files?

!kraken list #list of models 

# Connect to gdrive file directory    

````
from google.colab import drive
drive.mount('/content/drive')
````

FOLLOWING MAY NOT BE NECESSARY    
need to use following "magic" notation starting with %     

switch to gdrive files
````
%cd gdrive/MyDrive/kraken_project_folder/
# !pwd #print working directory
````
    
switch to colab "home"
````
%cd /content/
````



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#%cd /content/drive/MyDrive/kraken_project_folder/paris_BNF328-329
#%cd /content/drive/MyDrive/kraken_project_folder/maimonides_autograph
%cd /content/drive/MyDrive/kraken_project_folder/naples
!pwd

# Setup for recognition training
````
ls path/*.xml | sort -R > whatever.xmllist
cat whatever.xmllist | wc -l  
split whatever.xmllist -l num_lines
mv xaa whatever_train.xmllist
mv xab whatever_test.xmllist
````

In [ ]:
#path variables
project_name = "naples"
trial = "2021_05_14_1_"
recog_model = "sinai_no_voc_61.mlmodel"

In [ ]:
# get training data from already downloaded from escriptorium

!ls recognition_training_xml/*.xml | sort -R \
> recognition_training_xml/{project_name}_xml.txt

In [ ]:
# how many lines in xmllist?
!cat recognition_training_xml/{project_name}_xml.txt | wc -l 

Explicit test and training data

Set, e.g., 90% to training, and 10% to testing

In [ ]:
# In test case 18 lines in naples_xml.txt
# 90% = 16
!split recognition_training_xml/{project_name}_xml.txt -l 18
!mv xaa recognition_training_xml/{project_name}_xml_train.txt
!mv xab recognition_training_xml/{project_name}_xml_test.txt
!ls recognition_training_xml/

In [ ]:
# verify that there
!ls recognition_training_xml/{project_name}_xml.txt

# Recognition training with kraken


##Train from scratch   
Next cell: no upload of existing model

In [ ]:
!ketos -v train  \
-o trained_recognition_models/{project_name}_scratch_2 \
-t recognition_training_xml/{project_name}_xml.txt \
-q early \
--normalize-whitespace \
--reorder \
-f alto \
--partition 0.9 \
-d cuda:0


##Training on top of existing model

-i or --load : loads existing model   
--force binarization [recommendation of kraken; throws an error]  
--append: remove layers from original model [need to learn how to apply]

In [ ]:
!ketos train \
-o trained_recognition_models/{project_name}_{trial} \
-t recognition_training_xml/{project_name}_xml_train.txt \
-e recognition_training_xml/{project_name}_xml_test.txt \
-q early \
--normalize-whitespace \
--reorder \
-f alto \
-d cuda:0 \
--resize add \
-i ../kraken/recognition_models/{recog_model}  \
-r 0.0001 \
-B 1 # batchsize
# --append 7
# --force-binarization throws an error


!ketos test -e recognition_training_xml/naples_xml_test.txt -

trained_recognition_models/{project_name}_{trial}_best.mlmodel


In [ ]:
!ketos test \
-f xml \
-e recognition_training_xml/{project_name}_xml.txt \
-m trained_recognition_models/{project_name}_{trial}_best.mlmodel